In [312]:
import pickle
import pandas as pd
from bokeh.plotting import figure
from bokeh.io import show, output_notebook, push_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.models import FactorRange

from bokeh.layouts import row, column
from bokeh.models.widgets import Div
from bokeh.models import Title
from bokeh.models.widgets import Select
from ipywidgets import interactive
output_notebook()

Loading BokehJS ...

In [313]:
def f(x):
    return x

In [314]:
with open('palabras.pickle', 'rb') as handle:
    palabras =  pickle.load(handle)
with open('palabras_hist.pickle', 'rb') as handle:
    palabras_hist =  pickle.load(handle)

In [315]:
palabras_hist = palabras_hist.reset_index()

In [338]:
def obtain_row(grupo, pleno):
    x = palabras[(palabras['grupo'] == grupo) & (palabras['pleno'] == pleno)]['palabra']
    y = palabras[(palabras['grupo'] == grupo) & (palabras['pleno'] == pleno)]['numero_veces']
    source = ColumnDataSource(data={'x': x, 'y': y})
    p1 = figure(x_range=FactorRange(*x),height=250, width=300, 
                title='Palabras más repetidas', toolbar_location=None, active_drag=None, active_scroll=None)
    
    x = palabras[(palabras['grupo'] == grupo) & (palabras['pleno'] == pleno)]['palabra']
    y = palabras[(palabras['grupo'] == grupo) & (palabras['pleno'] == pleno)]['veces_acumuladas']
    source = ColumnDataSource(data={'x': x, 'y': y})
    p2 = figure(x_range=FactorRange(*x),height=250, width=300, 
                title='histórico acumulado', toolbar_location=None, active_drag=None, active_scroll=None)
    
    x = palabras_hist[(palabras_hist['grupo'] == grupo) & (palabras_hist['pleno'] == pleno)]['palabra']
    y = palabras_hist[(palabras_hist['grupo'] == grupo) & (palabras_hist['pleno'] == pleno)]['veces']
    source = ColumnDataSource(data={'x': x, 'y': y})
    p3 = figure(x_range=FactorRange(*x),height=250, width=300, 
                title='histórico total', toolbar_location=None, active_drag=None, active_scroll=None)
    
    p1.vbar(x='x', top='y', width=0.9, source=source, color='#FA4F00')
    p2.vbar(x='x', top='y', width=0.9, source=source, color='#FA4F00')
    p3.vbar(x='x', top='y', width=0.9, source=source, color='#FA4F00')
    
    p1.y_range.start = 0
    p1.x_range.range_padding = 0.1
    p1.xaxis.major_label_orientation = 1
    p1.xgrid.grid_line_color = None

    p2.y_range.start = 0
    p2.x_range.range_padding = 0.1
    p2.xaxis.major_label_orientation = 1
    p2.xgrid.grid_line_color = None

    p3.y_range.start = 0
    p3.x_range.range_padding = 0.1
    p3.xaxis.major_label_orientation = 1
    p3.xgrid.grid_line_color = None
    
    return row(p1, p2, p3)

In [339]:
def generate_data(grupo, pleno):
    show(column(obtain_row(grupo, pleno)), notebook_handle=True) 
    push_notebook()
   
    

In [340]:
grupos = palabras['grupo'].drop_duplicates()

In [341]:
plenos = palabras['pleno'].drop_duplicates()

In [342]:
interactive_plot = interactive(generate_data, grupo=grupos[0:2].to_list(), pleno=plenos[0:2].to_list())

In [343]:
interactive_plot

In [344]:
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

In [346]:
import yaml

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.io import show, output_notebook

from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature

output_notebook()

Loading BokehJS ...

def bkapp(doc):
    df = sea_surface_temperature.copy()
    source = ColumnDataSource(data=df)

    plot = figure(x_axis_label='datetime', y_range=(0, 25),
                  y_axis_label='Temperature (Celsius)',
                  title="Sea Surface Temperature at 43.18, -70.43")
    plot.line('time', 'temperature', source=source)

    def callback(attr, old, new):
        if new == 0:
            data = df
        else:
            data = df.rolling('{0}D'.format(new)).mean()
        source.data = ColumnDataSource.from_df(data)

    slider = Slider(start=0, end=30, value=0, step=1, title="Smoothing by N Days")
    slider.on_change('value', callback)

    doc.add_root(column(slider, plot))

    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """, Loader=yaml.FullLoader))

In [541]:
def bkapp(doc):
    
    grupo = 'Grupo Ciudadanos'
    pleno = '00_INVESTIDURA'
    
    x = palabras[(palabras['grupo'] == grupo) & (palabras['pleno'] == pleno)]['palabra']
    y = palabras[(palabras['grupo'] == grupo) & (palabras['pleno'] == pleno)]['numero_veces']
    source = ColumnDataSource(data={'palabra': x, 'numero_veces': y})
    

    
    plot = figure(x_range=FactorRange(*x), 
                  height=250, 
                  width=300,
                  toolbar_location=None, 
                  active_drag=None, 
                  active_scroll=None,
                  x_axis_label='palabra',
                  y_axis_label='veces repetida',
                  title=pleno)
    
    plot.vbar(x='palabra', top='numero_veces', width=0.9, source=source, color='#FA4F00')
    
    plot.xaxis.major_label_orientation = 1

    plenos_list = plenos.to_list()
    
    def callback(attr, old, new):
        if new == 0:
            grupo = 'Grupo Ciudadanos'
            pleno = '00_INVESTIDURA'
            data = palabras[(palabras['grupo'] == grupo) & (palabras['pleno'] == pleno)][['palabra', 'numero_veces']]
        else:
            grupo = 'Grupo Ciudadanos'
            pleno = plenos_list[new]
            data = palabras[(palabras['grupo'] == grupo) & (palabras['pleno'] == pleno)][['palabra', 'numero_veces']]
            
        source.data = ColumnDataSource.from_df(data)
        
        

    slider = Slider(start=0, end=len(plenos_list) - 1, value=0, step=1, title="Plenos")
    slider.on_change('value', callback)

    doc.add_root(column(slider, plot))

    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """, Loader=yaml.FullLoader))

In [542]:
show(bkapp, notebook_url="http://localhost:8889")

2020_02_04
2020_02_11
2020_02_12
2020_02_18
2020_02_19
2020_02_20
2020_02_25
2020_02_26
2020_02_27
2020_02_26
2020_02_27
2020_02_26
2020_02_25
2020_02_19
2020_02_18
2020_02_12
2020_02_11
2020_02_04


In [540]:
plenos

0    00_INVESTIDURA
0        2020_02_04
0        2020_02_11
0        2020_02_12
0        2020_02_18
0        2020_02_19
0        2020_02_20
0        2020_02_25
0        2020_02_26
0        2020_02_27
Name: pleno, dtype: object